# Imports de todas as bibliotecas utilizadas


In [1]:
!pip3 install --upgrade scikit-learn


## Imports de todas as bibliotecas utilizadas


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from google.colab import drive
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import seaborn as sns



## Leitura do DataFrame


In [ ]:

drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/Shareddrives/ProjetoAM/music-inRecomendation/

df = pd.read_csv('data/dados_treino.csv')

In [ ]:
df.columns

## Limpeza de dados

In [ ]:
# Limpeza de dados
df.query('duracao >0', inplace=True)
df.query('PctCantada <= 100 & PctCantada>=0',inplace=True)
df.query('PctRap <= 100 & PctRap>=0',inplace=True)



In [ ]:
#### TESTE!!!!!!!
# df = df.drop(columns=["BPM", "Tem_Instr_Guitarra", "Tem_Instr_Metais", "Tem_Instr_Madeiras"])
#df = df.drop(columns=["Tem_Instr_Metais"])

### Ajustes das variáveis

In [ ]:
# Substituindo os booleanos para 0 e 1
df["Tem_Instr_Violao_Viola"] = df["Tem_Instr_Violao_Viola"].astype(int)
df["Tem_Instr_Guitarra"] = df["Tem_Instr_Guitarra"].astype(int)
df["Tem_Instr_Cavaco"] = df["Tem_Instr_Cavaco"].astype(int)
df["Tem_Instr_Sintetizador_Teclado"] = df["Tem_Instr_Sintetizador_Teclado"].astype(int)
df["Tem_Instr_Piano"] = df["Tem_Instr_Piano"].astype(int)
df["Tem_Instr_Metais"] = df["Tem_Instr_Metais"].astype(int)
df["Tem_Instr_Madeiras"] = df["Tem_Instr_Madeiras"].astype(int)
df["Tem_Instr_Cordas"] = df["Tem_Instr_Cordas"].astype(int)
df["aovivo"] = df["aovivo"].astype(int)
df["explicita"] = df["explicita"].astype(int)
df["gostou"] = df["gostou"].astype(int)


In [ ]:
# Criando o campo da tonalidade maior
def maior(df):
    if df.maior == True:
         return "Maior"
    elif df.maior == False:
         return "Menor"
    else:
        return "NA"

df['maior'] = df['modo'].replace('#',"", regex=False)
df['maior'] = df['maior'].str.isupper()
df['maior'] = df.apply(maior, axis=1)


df['modo']=df['modo'].str.upper()

In [ ]:
# Criando o campo da década da música 
df['decada'] = (df['ano_lancamento'] // 10) * 10



In [ ]:
# Dropando Modo e Ano de Lançamento
df = df.drop(columns=["ano_lancamento"])

In [ ]:
users = df["id_cliente"]
print(users.unique())

# Separando DFs por user
df_user1 = df.query('id_cliente == "#ID7758"')
df_user2 = df.query('id_cliente == "#ID4820"')

In [ ]:
#df_user1["reproducao_normalizada"] = df_user1["n_reproducao"] / max(df_user1["n_reproducao"].values)
#df_user2["reproducao_normalizada"] = df_user2["n_reproducao"] / max(df_user2["n_reproducao"].values)

#df_user1 = df_user1.drop(columns=["n_reproducao"])
#df_user2 = df_user2.drop(columns=["n_reproducao"])

In [ ]:
# Separando os valores de X e Y
y_user1 = df_user1['gostou']
y_user2 = df_user2['gostou']

df_x_user1 = df_user1.drop(columns=["gostou", "id_cliente"])
df_x_user2 = df_user2.drop(columns=["gostou", "id_cliente"])

x_user1 = df_x_user1
x_user2 = df_x_user2

In [ ]:
# Aplicando Dummies (Alternativa a OneHotEncoder)
x_user1 = pd.get_dummies(data=x_user1, columns=["bateria", "maior"])
x_user1 = x_user1.drop(columns=["bateria_Nenhuma", "maior_NA",'maior_Menor']).rename({'maior_Maior':'maior'})

x_user2 = pd.get_dummies(data=x_user2, columns=["bateria", "maior"])
x_user2 = x_user2.drop(columns=["bateria_Nenhuma", "maior_NA",'maior_Menor']).rename({'maior_Maior':'maior'})



###### TESTE TESTE
# x_user1 = x_user1.drop(columns=["Tem_Instr_Metais", "Tem_Instr_Cordas", "bateria_Acústica", "bateria_Eletrônica", "Tem_Instr_Sintetizador_Teclado"])
# x_user2 = x_user2.drop(columns=["Tem_Instr_Metais", "Tem_Instr_Cordas", "bateria_Acústica", "bateria_Eletrônica", "Tem_Instr_Sintetizador_Teclado"])

### OneHot SKLEARN

In [ ]:
# SKLEAN
## EXPLICAR O PQ ESTAMOS TRANSFORMANDO PARA NUMEROS: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

# labelencoder = LabelEncoder()

# x_user1[:, 0] = labelencoder.fit_transform(x_user1[:, 0])
# x_user2[:, 0] = labelencoder.fit_transform(x_user2[:, 0])

# x_user1[:, 16] = labelencoder.fit_transform(x_user1[:, 16])
# x_user2[:, 16] = labelencoder.fit_transform(x_user2[:, 16])

## onehotencoder da coluna bateria
## EXPLICAR O PQ DIMINUIMOS A DIMENSAO

# ct = ColumnTransformer([("bateria", OneHotEncoder(), [0])], remainder = 'passthrough')

# x_user1 = ct.fit_transform(x_user1)
# x_user1 = x_user1[:, 1:]

# x_user2 = ct.fit_transform(x_user2)
# x_user2 = x_user2[:, 1:]

labelencoder = LabelEncoder()

x_user1['modo'] = labelencoder.fit_transform(x_user1['modo'])
x_user2['modo'] = labelencoder.fit_transform(x_user2['modo'])


### Scaler

In [ ]:
x_user1.columns

In [ ]:
# Aplicando StandardScaler



#scalerUser1 = StandardScaler()
#scalerUser2 = StandardScaler()

#scalerUser1.fit(x_user1)
#scalerUser2.fit(x_user2)

## Separação treino e teste
###  - explicar no relatório o stratify e shuffle

SEED = 5
np.random.seed(SEED)

x_train_user1, x_test_user1, y_train_user1, y_test_user1 = train_test_split(x_user1, y_user1, test_size = 0.2,shuffle=True, stratify=y_user1, random_state=42)

x_train_user2, x_test_user2, y_train_user2, y_test_user2 = train_test_split(x_user2, y_user2, test_size = 0.2,shuffle=True, stratify=y_user2, random_state=42)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


cols_to_scale = ['BPM', 'VolMedio','PctCantada','PctRap','duracao','n_reproducao','decada']

scaler = StandardScaler()
scaler.fit(x_train_user1[cols_to_scale])
x_train_user1[cols_to_scale] = scaler.transform(x_train_user1[cols_to_scale])

scaler = StandardScaler()
scaler.fit(x_train_user2[cols_to_scale])
x_train_user2[cols_to_scale] = scaler.transform(x_train_user2[cols_to_scale])

scaler = StandardScaler()
scaler.fit(x_user1[cols_to_scale])
x_test_user1[cols_to_scale] = scaler.transform(x_test_user1[cols_to_scale])

scaler = StandardScaler()
scaler.fit(x_test_user2[cols_to_scale])
x_test_user2[cols_to_scale] = scaler.transform(x_test_user2[cols_to_scale])



In [ ]:
#x_train_user1= x_train_user1.drop(columns=['decada', 'modo'])
#x_test_user1=x_test_user1.drop(columns=['decada', 'modo'])

#x_train_user2=x_train_user2.drop(columns=['decada', 'modo'])
#x_test_user2=x_test_user2.drop(columns=['decada', 'modo'])

In [ ]:
## Scaler dos valores X após separar treino e teste 

# user 1
#x_train_user1 = scalerUser1.transform(x_train_user1)
#x_test_user1 = scalerUser1.transform(x_test_user1)


# user 2
#x_train_user2 = scalerUser2.transform(x_train_user2)
#x_test_user2 = scalerUser2.transform(x_test_user2)


In [ ]:
# Função Dummy
def dummy(x_train, x_test, y_train, y_test):
	dummy_stratified = DummyClassifier(strategy="stratified")
	dummy_stratified.fit(x_train, y_train)
 
	acuracia = dummy_stratified.score(x_test, y_test) * 100

	print("A acurácia do algoritmo [DUMMY] foi %.2f%%" % acuracia)
	return acuracia

In [ ]:
# Função Random Forest
def randomforest(x_train, x_test, y_train, y_test, n_estimators=1000):
	modelo = RandomForestClassifier(n_estimators=n_estimators)
	modelo.fit(x_train, y_train)
	modelo.score(x_test, y_test)
 
	previsoes = modelo.predict(x_test)
	acuracia = accuracy_score(y_test, previsoes) * 100
  
	print("A acurácia do algoritmo [RANDOM FOREST] foi %.2f%%" % acuracia)

	return (modelo, previsoes, acuracia)

In [ ]:
# Função SVC
def svc(x_train, x_test, y_train, y_test):
  modelo = SVC()
  modelo.fit(x_train, y_train)
   utilizando os 
  previsoes = modelo.predict(x_test)
  acuracia = accuracy_score(y_test, previsoes) * 100

  print("A acurácia do algoritmo [SVC] foi %.2f%%" % acuracia)

  return (modelo, previsoes, acuracia)

In [ ]:
# Função Regressão logistíca
def logistic_regression(x_train, x_test, y_train, y_test):
  modelo = LogisticRegression(random_state=42)
  modelo.fit(x_train, y_train)
  modelo.score(x_test, y_test)

  previsoes = modelo.predict(x_test)
  acuracia = accuracy_score(y_test, previsoes) * 100

  print("A acurácia do algoritmo [LOGISTIC REGRESSION] foi %.2f%%" % acuracia)

  return (modelo, previsoes, acuracia)

In [ ]:
# Função árvore de decisão
def decision_tree_fn(x_train, x_test, y_train, y_test, max_range=15):
  test=[]
  train=[]

  for i in range(1, max_range):
      modelo = DecisionTreeClassifier(max_depth = i)
      modelo.fit(x_train, y_train)
      test.append(modelo.score(x_test, y_test))
      train.append(modelo.score(x_train, y_train))


  print("A acurácia do algoritmo [DECISION TREE] foi %.2f%%" % (max(test) * 100))
  modelo = DecisionTreeClassifier(max_depth = test.index(max(test)))
  modelo.fit(x_train, y_train)
  previsoes = modelo.predict(x_test)
  acuracia = accuracy_score(y_test, previsoes) * 100
  return (modelo, previsoes, acuracia)
  


In [ ]:
## Printando valores para todos os cenários
print("USER 1\n")
dummy_user1 = dummy(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)
rf_user1 = randomforest(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)
svc_user1 = svc(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)
logistic_user1 = logistic_regression(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)
decision_user1 = decision_tree_fn(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)
print("------------------------\n")

print("USER 2\n")
dummy_user2 = dummy(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)
rf_user2 = randomforest(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)
svc_user2 = svc(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)
logistic_user2 = logistic_regression(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)
decision_user2 = decision_tree_fn(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)
print("------------------------\n")

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.backend import clear_session

# model = Sequential()
# model.add(Dense(32, input_dim=21, activation='tanh'))
# model.add(Dense(80, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(2, activation='softmax'))

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(x_train_user1, y_train_user1, epochs=150, batch_size=10)

# _, accuracy = model.evaluate(x_test_user1, y_test_user1)
# print('Accuracy: %.2f' % (accuracy*100))

# clear_session()

In [ ]:
# MLP com SKLearn
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(1, 30), random_state=1)
clf.fit(x_train_user1, y_train_user1)               

previsoes = clf.predict(x_test_user1)
acuracia = accuracy_score(y_test_user1, previsoes) * 100

print("A acurácia do algoritmo [MLP] foi %.2f%%" % acuracia)

In [ ]:
# MLP com SKLearn

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(1, 30), random_state=1)
clf.fit(x_train_user2, y_train_user2)               

previsoes = clf.predict(x_test_user2)
acuracia = accuracy_score(y_test_user2, previsoes) * 100

print("A acurácia do algoritmo [MLP] foi %.2f%%" % acuracia)

In [ ]:
previsoes

In [ ]:
val, _, _ = logistic_user1

result = permutation_importance(
    val, x_test_user1, y_test_user1, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=x_user1.columns[sorted_idx])
ax.set_title("Permutation Importance LR")
fig.tight_layout()
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/permplotLR_user1.pdf', dpi=150)

plt.show()


val, _, _ = decision_user1

result = permutation_importance(
    val, x_test_user1, y_test_user1, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=x_user1.columns[sorted_idx])
ax.set_title("Permutation Importance DT")
fig.tight_layout()
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/permplotSVC_user1.pdf', dpi=150)

plt.show()

In [ ]:
val, _, _ = logistic_user2

result = permutation_importance(
    val, x_test_user2, y_test_user2, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=x_user2.columns[sorted_idx])
ax.set_title("Permutation Importance LR")
fig.tight_layout()
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/permplotLR_user2.pdf', dpi=150)

plt.show()

val, _, _ = decision_user2

result = permutation_importance(
    val, x_test_user2, y_test_user2, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(
    result.importances[sorted_idx].T, vert=False, labels=x_user2.columns[sorted_idx])
ax.set_title("Permutation Importance DT")
fig.tight_layout()
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/permplotSVC_user2.pdf', dpi=150)

plt.show()

In [ ]:
##user1
modelo_lr1, previsao_lr1, acuracia_lr1 = logistic_regression(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)
modelo_svc1, previsao_svc1, acuracia_svc1 = svc(x_train=x_train_user1, x_test=x_test_user1, y_train=y_train_user1, y_test=y_test_user1)

##user2
modelo_lr2, previsao_lr2, acuracia_lr2 = logistic_regression(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)
modelo_svc2, previsao_svc2, acuracia_svc2 = svc(x_train=x_train_user2, x_test=x_test_user2, y_train=y_train_user2, y_test=y_test_user2)


### Visualização de métricas de desempenho

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

#user1
lr_disp=metrics.plot_roc_curve(modelo_lr1, x_test_user1, y_test_user1)
svc_disp=metrics.plot_roc_curve(modelo_svc1, x_test_user1, y_test_user1,ax=lr_disp.ax_)
svc_disp.figure_.suptitle("Comparação curvas ROC - usuário ID7758 ")
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/ROC_user1.pdf', dpi=150)
plt.show()
#user2
lr_disp=metrics.plot_roc_curve(modelo_lr2, x_test_user2, y_test_user2)
svc_disp=metrics.plot_roc_curve(modelo_svc2, x_test_user2, y_test_user2,ax=lr_disp.ax_) 
svc_disp.figure_.suptitle("Comparação curvas ROC - usuário ID4820 ")
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/ROC_user2.pdf', dpi=150)
plt.show()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(modelo_lr1, x_test_user1, y_test_user1, normalize='pred')
plt.suptitle('Matriz de confusão LR - usuário ID7758')
#plt.title('Normalização pelas predições realizadas (colunas)')
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/CF_LR_user1.pdf', dpi=150)

plt.show()

In [ ]:
plot_confusion_matrix(modelo_svc1, x_test_user1, y_test_user1, normalize='pred')
plt.suptitle('Matriz de confusão SVC - usuário ID7758')
#plt.title('Normalização pelas predições realizadas (colunas)')
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/CF_SVC_user1.pdf', dpi=150)

plt.show()

In [ ]:
plot_confusion_matrix(modelo_lr2, x_test_user2, y_test_user2, normalize='pred')
plt.suptitle('Matriz de confusão LR - usuário ID4820')
#plt.title('Normalização pelas predições realizadas (colunas)')
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/CF_LR_user2.pdf', dpi=150)

plt.show()

In [ ]:
plot_confusion_matrix(modelo_svc2, x_test_user2, y_test_user2, normalize='pred')
plt.suptitle('Matriz de confusão SVC - usuário ID4820')
#plt.title('Normalização pelas predições realizadas (colunas)')
plt.savefig('/content/drive/Shareddrives/ProjetoAM/music-inRecomendation/outputs/CF_SVC_user2.pdf', dpi=150)

plt.show()